[Rendered Version](http://nbviewer.jupyter.org/github/MaxGhenis/taxcalc-notebooks/blob/master/taxcalc%20Training%20Doc%20-%20TCJA%20-%20Tedeschi%20weights.ipynb)

This resembles the [normal training notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/Training%20Doc.ipynb), with the following changes:
* Models the House TCJA reform, not the sample reform.
* Uses Ernie Tedeschi's weights instead. This approach can be used to use any other weight file.
* Removes reference to sample reform dictionary.

In [1]:
from taxcalc import *

This example uses the packaged CPS file. If you were to use the PUF, you would create the records class using
`Records()` or `Records('path to puf')` if the PUF is not in your current working directory.

In [2]:
# Load Ernie Tedeschi's weights file.
tedeschi_weights = pd.read_csv('cps_weights_2026.csv')

In [3]:
# Initiate baseline calculator
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
# Replace weights attribute of calc.
calc.records.s006 = tedeschi_weights.s006
# Advance and calculate.
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [4]:
# Initiate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())
# Replace weights attribute of calc_x.
calc_x.records.s006 = tedeschi_weights.s006

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


You can specify your reforms in a JSON file and, using the `read_json_param_objects` method, create a dictionary containing both policy reforms and behavioral assumptions

The policy reform and any behavioral assumptions need to be in separate JSON files.

Here is [more information](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/reforms/REFORMS.md) on creating reform files.

In [5]:
reforms = calc_x.read_json_param_objects('TCJA_Nov3.json', 'SampleBehavior.json')
reforms

{'behavior': {2018: {u'_BE_inc': [-0.25]}},
 'consumption': {2018: {u'_MPC_e19800': [0.2]}},
 'growdiff_baseline': {},
 'growdiff_response': {},
 'policy': {2018: {u'_ALD_DomesticProduction_hc': [1],
   u'_ALD_StudentLoan_hc': [1],
   u'_ALD_Tuition_hc': [1],
   u'_AMT_rt1': [0],
   u'_AMT_rt2': [0],
   u'_CG_brk1': [[38600, 77200, 38600, 51700, 77200]],
   u'_CG_brk2': [[425800, 479000, 239500, 452400, 479000]],
   u'_CR_SchR_hc': [1],
   u'_CTC_c': [1100],
   u'_CTC_ps': [[115000, 230000, 115000, 115000, 115000]],
   u'_DependentCredit_Child_c': [500],
   u'_DependentCredit_Nonchild_c': [300],
   u'_FilerCredit_c': [[300, 600, 300, 300, 300]],
   u'_ID_Casualty_hc': [1],
   u'_ID_Medical_hc': [1],
   u'_ID_Miscellaneous_hc': [1],
   u'_ID_RealEstate_c': [[10000, 10000, 10000, 10000, 10000]],
   u'_ID_StateLocalTax_hc': [1],
   u'_ID_prt': [0],
   u'_II_brk1': [[45000, 90000, 45000, 67500, 90000]],
   u'_II_brk2': [[200000, 260000, 130000, 200000, 260000]],
   u'_II_brk3': [[500000, 1

In [6]:
calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [7]:
calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)

### Viewing the results

In [8]:
from taxcalc.utils import *

#### Analyzing Individual Variables

Individual variables are attributes of the records class and can therefore be accessed using a simple dot notation

In [9]:
baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2459274184595.68
-----------------------------------------------
Difference:                     -61796326853.96


#### Diagnostic Table

Diagnostic tables are the most straight forward methods of evaluation. They simply show aggregate values for a given calculator

In [10]:
create_diagnostic_table(calc)

,2018
Returns (#m),169.9
AGI ($b),"10,503.8"
Itemizers (#m),68.0
Itemized Deduction ($b),"1,500.9"
Standard Deduction Filers (#m),83.9
Standard Deduction ($b),771.4
Personal Exemption ($b),"1,222.6"
Taxable Income ($b),"7,421.9"
Regular Tax ($b),"1,369.7"
AMT Income ($b),"9,664.5"


In [11]:
create_diagnostic_table(calc_response)

,2018
Returns (#m),169.9
AGI ($b),"10,516.9"
Itemizers (#m),13.3
Itemized Deduction ($b),312.5
Standard Deduction Filers (#m),138.7
Standard Deduction ($b),"2,380.3"
Personal Exemption ($b),0.0
Taxable Income ($b),"8,175.3"
Regular Tax ($b),"1,408.1"
AMT Income ($b),"10,254.1"


#### Distribution Table

The distribution table shows the same information as the diagnostic table, but broken down by income bin or decile. You can view the results as either the weighted average or the weighted sum in each bin

In [12]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_avg')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,417",-662,1,"6,587",0,5,"5,262",24,1,-664,...,0,1,0,0,98,-97,176,79,-233,-312
1,"16,988,286","5,118",1,"7,814",0,207,"5,904",190,13,"5,018",...,0,13,3,0,755,-745,663,-82,"9,847","9,929"
2,"16,990,004","9,116",1,"7,860",0,"1,045","6,319","1,636",153,"8,514",...,1,154,29,0,"1,020",-895,"1,209",314,"17,245","16,931"
3,"16,989,656","17,018",1,"7,632",0,"2,335","7,151","5,057",532,"15,624",...,2,534,90,0,"1,193",-750,"2,327","1,577","25,262","23,685"
4,"16,989,955","26,269",1,"7,157",0,"3,910","7,667","10,826","1,244","23,898",...,4,"1,247",168,0,921,158,"3,609","3,768","34,401","30,633"
5,"16,989,241","37,913",1,"6,375",0,"5,941","8,090","19,220","2,298","34,325",...,5,"2,303",279,0,426,"1,598","5,182","6,780","45,965","39,185"
6,"16,989,686","54,723",0,"4,842",1,"9,622","8,499","32,350","4,206","48,877",...,3,"4,209",394,0,82,"3,733","7,416","11,149","61,857","50,708"
7,"16,989,485","78,655",0,"3,383",1,"13,995","9,343","52,170","7,544","70,468",...,4,"7,549",496,0,13,"7,040","10,754","17,794","85,684","67,890"
8,"16,989,613","114,892",0,"2,039",1,"19,742","10,530","82,679","12,939","103,702",...,6,"12,946",449,0,3,"12,494","15,860","28,354","124,252","95,898"
9,"16,989,497","275,212",0,990,1,"31,544","10,053","232,701","51,691","259,091",...,"1,036","52,726",29,549,0,"53,246","25,411","78,657","290,513","211,856"


In [13]:
create_distribution_table(calc_response.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,028","-9,827,556,603","9,579,064","191,135,893,552","4,231","3,273,315",0,"545,622,539","25,628,858","-9,830,829,918",...,0,"25,628,858","24,748,065",0,"1,662,090,300","-1,661,209,506","2,997,769,428","1,336,559,922","-3,958,098,965","-5,294,658,887"
1,"16,989,465","87,846,268,689","11,780,242","231,437,753,151","12,716","170,395,371",0,"1,524,778,468","67,851,552","87,703,049,456",...,0,"67,851,552","58,710,576",0,"12,902,375,287","-12,893,234,311","11,265,851,698","-1,627,382,613","167,236,434,997","168,863,817,610"
2,"16,989,636","155,821,082,556","13,342,978","243,003,031,103","96,983","1,413,697,034",0,"27,455,950,849","3,064,924,580","154,625,708,134",...,0,"3,064,924,580","2,074,937,200",0,"17,275,404,504","-16,285,417,124","20,539,812,804","4,254,395,679","292,686,742,734","288,432,347,055"
3,"16,989,794","289,944,850,143","15,522,265","263,968,108,365","233,657","3,330,455,031",0,"96,986,458,921","11,159,284,290","287,044,211,938",...,0,"11,159,284,290","5,124,614,055",0,"19,668,621,696","-13,633,951,462","39,495,697,139","25,861,745,677","428,957,361,721","403,095,616,044"
4,"16,989,434","447,028,410,765","16,123,368","275,700,316,814","471,983","6,744,736,802",0,"212,442,086,425","24,400,666,295","441,205,331,753",...,0,"24,400,666,295","9,571,249,753",0,"14,053,822,996","775,593,546","61,210,521,430","61,986,114,976","583,926,113,095","521,939,998,119"
5,"16,989,340","645,238,684,863","16,128,348","284,558,633,098","765,218","11,500,121,644",0,"373,003,913,970","42,895,014,156","635,353,140,182",...,0,"42,895,014,156","14,292,884,897",0,"5,693,139,146","22,908,990,114","87,989,026,304","110,898,016,417","779,979,327,503","669,081,311,085"
6,"16,988,819","929,566,823,028","15,918,877","294,144,465,502","1,060,763","17,282,806,808",0,"622,178,753,990","73,890,219,171","914,732,853,217",...,0,"73,890,219,171","18,966,028,687",0,"373,252,837","54,550,937,646","125,658,435,874","180,209,373,520","1,048,650,737,382","868,441,363,862"
7,"16,990,156","1,336,207,634,178","15,523,687","312,070,968,137","1,466,469","25,462,760,905",0,"998,984,689,236","131,572,917,131","1,314,512,275,772",...,0,"131,572,917,131","23,126,062,736",0,"25,206,925","108,421,647,470","182,305,299,494","290,726,946,964","1,453,078,946,395","1,162,351,999,430"
8,"16,989,672","1,951,791,397,145","14,487,192","314,244,583,720","2,502,480","54,135,802,517",0,"1,583,446,948,653","223,303,247,793","1,905,544,715,192",...,0,"223,303,247,793","26,398,389,436",0,"5,400,719","196,899,457,637","269,045,515,139","465,944,972,776","2,107,207,189,312","1,641,262,216,536"
9,"16,989,496","4,683,317,672,309","10,334,808","232,170,910,425","6,654,687","192,409,380,391",0,"4,258,737,381,493","897,692,010,275","4,523,228,824,225",...,0,"897,692,010,275","18,703,096,931","9,343,884,266",0,"888,332,797,610","431,350,643,666","1,319,683,441,276","4,931,895,622,582","3,612,212,181,306"


#### Differences Table

The differences table displays the difference between your baseline and refoms. You can also group the results by decile or income bin.

In [14]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58","107,203.82",0.63,"2,630.59",0.02,-0.58,"-9,928,489.71",0.02,-0.01,0.01
1,"16,988,285.70","1,913,087.92",11.26,"23,330.06",0.14,-14.55,"-247,179,274.80",0.40,-0.13,0.09
2,"16,990,004.05","5,425,493.22",31.93,"1,034,909.27",6.09,-72.74,"-1,235,777,171.30",2.00,-0.51,0.38
3,"16,989,656.05","7,490,780.81",44.09,"2,885,585.31",16.98,-63.38,"-1,076,860,206.74",1.74,-0.34,0.26
4,"16,989,954.66","8,845,717.18",52.06,"4,177,589.89",24.59,-130.57,"-2,218,411,482.41",3.59,-0.54,0.41
5,"16,989,240.80","10,499,711.16",61.80,"4,124,052.84",24.27,-237.27,"-4,031,045,204.05",6.52,-0.71,0.54
6,"16,989,686.38","11,963,599.35",70.42,"4,256,291.68",25.05,-538.20,"-9,143,775,370.17",14.80,-1.16,0.88
7,"16,989,484.52","12,247,440.35",72.09,"4,651,136.99",27.38,-664.10,"-11,282,708,373.45",18.26,-1.04,0.79
8,"16,989,613.44","12,108,717.63",71.27,"4,872,298.39",28.68,-935.53,"-15,894,347,292.05",25.72,-1.02,0.78
9,"16,989,497.48","10,603,706.01",62.41,"6,385,753.82",37.59,-980.39,"-16,656,293,989.29",26.95,-0.52,0.40


#### Plotting

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [15]:
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [16]:
mtr_plot_data = mtr_graph_data(calc, calc_response)

In [17]:
show(xtr_graph_plot(mtr_plot_data))

In [20]:
atr_plot_data = atr_graph_data(calc, calc_response)

In [21]:
show(xtr_graph_plot(atr_plot_data))

#### Multiyear diagnostic tables

You can also produce diagnostic tables up to 2026

In [22]:
multiyear_diagnostic_table(calc, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,503.8","10,987.4","11,474.4","11,999.4","12,561.1","13,146.7","13,759.3","14,401.2","15,070.1"
Itemizers (#m),68.0,69.6,71.2,72.7,74.2,75.9,77.5,79.1,80.8
Itemized Deduction ($b),"1,500.9","1,585.9","1,672.2","1,765.4","1,863.5","1,967.0","2,075.8","2,189.3","2,312.4"
Standard Deduction Filers (#m),83.9,84.3,84.8,85.4,86.0,86.5,87.1,87.7,88.3
Standard Deduction ($b),771.4,791.5,813.9,838.1,863.3,887.9,914.4,940.9,968.1
Personal Exemption ($b),"1,222.6","1,266.2","1,312.4","1,361.2","1,412.5","1,464.9","1,519.7","1,576.1","1,634.7"
Taxable Income ($b),"7,421.9","7,768.6","8,113.2","8,485.3","8,886.3","9,305.2","9,742.7","10,203.3","10,678.8"
Regular Tax ($b),"1,369.7","1,442.9","1,514.0","1,590.9","1,672.5","1,756.7","1,843.9","1,935.7","2,029.1"
AMT Income ($b),"9,664.5","10,099.5","10,536.6","11,008.0","11,513.5","12,039.3","12,589.2","13,165.8","13,763.9"


In [23]:
multiyear_diagnostic_table(calc_response, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,516.9","11,001.2","11,488.8","12,014.5","12,577.0","13,163.4","13,776.8","14,419.6","15,089.5"
Itemizers (#m),13.3,14.2,15.2,16.2,17.2,18.2,19.3,20.3,21.5
Itemized Deduction ($b),312.5,346.4,381.2,419.1,459.6,503.0,550.1,599.7,655.5
Standard Deduction Filers (#m),138.7,139.8,141.0,142.1,143.3,144.4,145.5,146.7,147.7
Standard Deduction ($b),"2,380.3","2,449.7","2,523.9","2,601.8","2,683.8","2,766.1","2,851.2","2,938.4","3,026.3"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"8,175.3","8,565.8","8,954.5","9,374.8","9,826.3","10,298.4","10,791.8","11,310.1","11,848.6"
Regular Tax ($b),"1,408.1","1,484.8","1,559.6","1,640.7","1,726.5","1,814.9","1,906.6","2,003.0","2,101.8"
AMT Income ($b),"10,254.1","10,709.7","11,167.8","11,661.3","12,189.6","12,739.1","13,312.6","13,913.4","14,536.0"


#### Reporting your changes

To display what reforms you included in a way that is easy for humans to read, you can use the `reform_documentation` method. It will print out all of the policy parameters you've specified with a short description and their default and current values


_Note: this feature is not yet available in the taxcalc package. You must use the source code to access it. It will be available in the next taxcalc release._

In [24]:
print(calc_response.reform_documentation(reforms))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _ALD_DomesticProduction_hc : 1
  name: Deduction for domestic production activity haircut
  desc: If greater than zero, this decimal fraction reduces the portion of
        domestic production activity that can be deducted from AGI.
  baseline_value: 0.0
 _ALD_StudentLoan_hc : 1
  name: Adjustment for student loan interest haircut
  desc: This decimal fraction can be applied to limit the student loan
        interest adjustment allowed.
  baseline_value: 0.0
 _ALD_Tuition_hc : 1
  name: Deduction for tuition and fees haircut
  desc: If greater than zero, this decimal fraction reduces the portion of
        tuition and fees that can be deducted from AGI.
  baseline_value: 0.0
 _AMT_rt1 : 0
  name: AMT rate 1
  desc: The tax rate applied to the portion of AMT taxable income below the
        surtax threshold, AMT bracket

One thing I didn't cover is how to extract the marginal tax rates. All you need to do is call the `Calculator.mtr()` method. It will return MTR for individual income tax, payroll tax, and individaul income tax + payroll tax

In [25]:
mtr_payroll, mtr_income, mtr_combined = calc.mtr()

In [26]:
mtr_payroll

array([ 0.14212726,  0.14212726,  0.14212726, ...,  0.14212726,
        0.14212726,  0.14212726])

In [27]:
mtr_income

array([ 0.13934046,  0.        ,  0.        , ...,  0.23687877,
        0.23687877,  0.23687877])

In [28]:
mtr_combined

array([ 0.28146772,  0.14212726,  0.14212726, ...,  0.37900604,
        0.37900604,  0.37900604])